In [1]:
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Carregar os dados
gdf = gpd.read_file("..\..\Data\Processed\PT-FireSprd_v2.1\L2_FireBehavior\PT-FireProg_v2.1_L2_model.shp")

print("Primeiras linhas do dataset:")
print(gdf.head())
print("\nInformações do dataset:")
print(gdf.info())
print("\nColunas disponíveis:")
print(gdf.columns.tolist())

Primeiras linhas do dataset:
       inidoy      enddoy        ros_p  duration_p     elev_av   aspect_av  \
0  221.750000  221.958333  1199.641053         5.0  222.993924  184.797777   
1  221.958333  222.125000   397.497644         4.0  167.295794  164.945149   
2  222.500000  222.604167  1092.753836         2.5  211.103842  248.382782   
3  222.604167  222.916667   584.937417         7.5  184.752690  152.575136   
4  241.722222  241.784722   708.321556         1.5  170.930441  276.002661   

   landform  land_use  1_3y_fir_p  3_8y_fir_p  ...  LCL_m_av  LFC_hPa_av  \
0      21.0       4.0         0.0         0.0  ...  2.187129         NaN   
1      22.0       4.0         0.0         0.0  ...  1.955263         NaN   
2      21.0       4.0         0.0         0.0  ...  3.322814         NaN   
3      21.0       4.0         0.0         0.0  ...  2.860403         NaN   
4      21.0       4.0         0.0         0.0  ...  1.686957         NaN   

   CCL_hPa_av  EL_m_av  LiftIdx_av    VentIdx

In [2]:
# Separar os dados
grupo_alto = gdf[gdf['ros_p'] > 2000]
grupo_medio = gdf[(gdf['ros_p'] > 1000) & (gdf['ros_p'] <= 2000)]
grupo_baixo = gdf[gdf['ros_p'] <= 1000]

print(f"\nGrupo com ros_p > 2000: {len(grupo_alto)} registros")
print(f"Grupo com 1000 < ros_p <= 2000: {len(grupo_medio)} registros")
print(f"Grupo com ros_p <= 1000: {len(grupo_baixo)} registros")

# Identificar colunas numéricas para análise
colunas_numericas = gdf.select_dtypes(include=[np.number]).columns.tolist()

# Remover colunas que não são relevantes para análise (como coordenadas, IDs, etc)
colunas_nao_relevantes = ['geometry', 'FID', 'OBJECTID', 'ID', 'id']
colunas_analise = [col for col in colunas_numericas if col not in colunas_nao_relevantes and col != 'ros_p']

print(f"\nColunas para análise: {colunas_analise}")


Grupo com ros_p > 2000: 132 registros
Grupo com 1000 < ros_p <= 2000: 196 registros
Grupo com ros_p <= 1000: 845 registros

Colunas para análise: ['inidoy', 'enddoy', 'duration_p', 'elev_av', 'aspect_av', 'landform', 'land_use', '1_3y_fir_p', '3_8y_fir_p', '8_ny_fir_p', 'fuel_model', 'f_load_av', 'sW_1m_av', 'sW_3m_av', 'sW_7_av', 'sW_28_av', 'sW_100_av', 'sW_289_av', 't_2m_C_av', 'd_2m_C_av', 'rh_2m_av', 'VPD_Pa_av', 'sP_hPa_av', 'gp_m2s2_av', 'dfmc_av', 'HDW_av', 'Haines_av', 'FWI_12h_av', 'DC_12h_av', 'FFMC_12h_a', 'wv10_kh_av', 'wdir10_av', 'wv100_k_av', 'wdir100_av', 'Recirc', 'CircVar', 't_950_av', 't_850_av', 't_700_av', 't_500_av', 't_300_av', 'rh_950_av', 'rh_850_av', 'rh_700_av', 'rh_500_av', 'rh_300_av', 'wv_950_av', 'wv_850_av', 'wv_700_av', 'wv_500_av', 'wv_300_av', 'wdi_950_av', 'wdi_850_av', 'wdi_700_av', 'wdi_500_av', 'wdi_300_av', 'vwv_950_av', 'vwv_850_av', 'vwv_700_av', 'vwv_500_av', 'vwv_300_av', 'gp_950_av', 'gp_850_av', 'gp_700_av', 'gp_500_av', 'gp_300_av', 'gT_

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy import stats
from matplotlib.backends.backend_pdf import PdfPages

# Primeiro, vamos garantir que temos os dados carregados corretamente
print("Preparando dados para análise...")

# Separar os dados
grupo_alto = gdf[gdf['ros_p'] > 2000]
grupo_baixo = gdf[gdf['ros_p'] <= 2000]

print(f"\nGrupo com ros_p > 2000: {len(grupo_alto)} registros")
print(f"Grupo com ros_p <= 2000: {len(grupo_baixo)} registros")

# Identificar colunas numéricas para análise
colunas_numericas = gdf.select_dtypes(include=[np.number]).columns.tolist()

# Remover colunas que não são relevantes para análise (como coordenadas, IDs, etc)
colunas_nao_relevantes = ['geometry', 'FID', 'OBJECTID', 'ID', 'id']
colunas_analise = [col for col in colunas_numericas if col not in colunas_nao_relevantes and col != 'ros_p']

print(f"\nColunas para análise: {len(colunas_analise)}")

# REALIZAR OS TESTES ESTATÍSTICOS PRIMEIRO
print("\n" + "="*50)
print("REALIZANDO TESTES ESTATÍSTICOS")
print("="*50)

resultados_teste = []

for coluna in colunas_analise:
    try:
        # Remover valores NaN
        dados_alto = grupo_alto[coluna].dropna()
        dados_baixo = grupo_baixo[coluna].dropna()
        
        if len(dados_alto) < 3 or len(dados_baixo) < 3:
            resultados_teste.append({
                'Variável': coluna,
                'Teste': 'Não aplicável',
                'Estatística': np.nan,
                'p-valor': np.nan,
                'Significativo': False
            })
            continue
            
        # Teste de normalidade (Shapiro-Wilk)
        try:
            _, p_alto = stats.shapiro(dados_alto)
            _, p_baixo = stats.shapiro(dados_baixo)
            
            normal_alto = p_alto > 0.05
            normal_baixo = p_baixo > 0.05
            
            # Escolher teste baseado na normalidade
            if normal_alto and normal_baixo:
                # Teste t para amostras independentes
                estatistica, p_valor = stats.ttest_ind(dados_alto, dados_baixo, equal_var=False)
                teste_usado = "Teste t (Welch)"
            else:
                # Teste de Mann-Whitney U (não paramétrico)
                estatistica, p_valor = stats.mannwhitneyu(dados_alto, dados_baixo)
                teste_usado = "Mann-Whitney U"
                
        except:
            # Se houver erro no teste paramétrico, usar não paramétrico
            estatistica, p_valor = stats.mannwhitneyu(dados_alto, dados_baixo)
            teste_usado = "Mann-Whitney U"
        
        resultados_teste.append({
            'Variável': coluna,
            'Teste': teste_usado,
            'Estatística': estatistica,
            'p-valor': p_valor,
            'Significativo': p_valor < 0.05
        })
        
    except Exception as e:
        print(f"Erro na variável {coluna}: {e}")
        resultados_teste.append({
            'Variável': coluna,
            'Teste': 'Erro',
            'Estatística': np.nan,
            'p-valor': np.nan,
            'Significativo': False
        })

print("Testes estatísticos concluídos!")

# AGORA CRIAR O RESUMO E OS GRÁFICOS
print("\n" + "="*80)
print("CRIANDO RESUMO E VISUALIZAÇÕES")
print("="*80)

# Criar DataFrame com resultados
resultados = []
for coluna in colunas_analise:
    # Encontrar os resultados correspondentes
    for resultado in resultados_teste:
        if resultado['Variável'] == coluna:
            try:
                resultados.append({
                    'Variável': coluna,
                    'Grupo Alto (média)': grupo_alto[coluna].mean(),
                    'Grupo Baixo (média)': grupo_baixo[coluna].mean(),
                    'Diferença': grupo_alto[coluna].mean() - grupo_baixo[coluna].mean(),
                    'p-valor': resultado['p-valor'],
                    'Significativo': resultado['Significativo'],
                    'Teste': resultado['Teste']
                })
            except:
                continue
            break

df_resumo = pd.DataFrame(resultados)

# Ordenar por significância e magnitude da diferença
df_resumo['abs_diferenca'] = abs(df_resumo['Diferença'])
df_resumo = df_resumo.sort_values(['Significativo', 'abs_diferenca'], ascending=[False, False])

print("RESUMO DAS VARIÁVEIS MAIS IMPORTANTES (SIGNIFICATIVAS)")
print("="*80)

# Filtrar apenas variáveis significativas
df_significativas = df_resumo[df_resumo['Significativo'] == True].head(30)

print(f"\nVariáveis com diferenças estatisticamente significativas (p < 0.05): {len(df_significativas)}")
print("\nTop 20 variáveis mais importantes:")
print(df_significativas[['Variável', 'Grupo Alto (média)', 'Grupo Baixo (média)', 'Diferença', 'p-valor']].head(20).round(4).to_string(index=False))

# VISUALIZAÇÕES PRINCIPAIS
print("\n" + "="*80)
print("CRIANDO VISUALIZAÇÕES")
print("="*80)

plt.style.use('default')
sns.set_palette("husl")

print("\n1. Gráfico de TODAS as variáveis — primeiro significativas, depois não significativas")

# Criar PDF para guardar os gráficos
pdf_path = r'..\..\Data\Data_Exploration\variaveis_diferentes_2000_ros.pdf'
pdf = PdfPages(pdf_path)

# Separar variáveis significativas e não significativas
df_signif = df_resumo[df_resumo['Significativo'] == True].sort_values('abs_diferenca', ascending=False)
df_nsignif = df_resumo[df_resumo['Significativo'] == False].sort_values('abs_diferenca', ascending=False)

# SIGNIFICATIVAS primeiro, mas dentro do grupo em ordem alfabética
df_signif_plot = df_signif.sort_values("Variável")

# NÃO significativas depois, também em ordem alfabética
df_nsignif_plot = df_nsignif.sort_values("Variável")

# Agora concatena mantendo a ordem desejada
df_plot = pd.concat([df_signif_plot, df_nsignif_plot], axis=0)


n = len(df_plot)
cols = 4
rows = (n // cols) + (1 if n % cols != 0 else 0)

fig, axes = plt.subplots(rows, cols, figsize=(20, 5*rows))
axes = axes.ravel()

for i, (idx, row) in enumerate(df_plot.iterrows()):
    var = row['Variável']

    try:
        dados_alto_clean = grupo_alto[var].dropna()
        dados_baixo_clean = grupo_baixo[var].dropna()

        dados = pd.DataFrame({
            'Valor': pd.concat([dados_alto_clean, dados_baixo_clean]),
            'Grupo': ['ros_p > 2000'] * len(dados_alto_clean) + ['ros_p ≤ 2000'] * len(dados_baixo_clean)
        })

        sns.boxplot(
            x='Grupo',
            y='Valor',
            data=dados,
            ax=axes[i],
            palette={'ros_p > 2000': 'red', 'ros_p ≤ 2000': 'blue'}
        )

        titulo = (
            f"{var}\n(p = {row['p-valor']:.4f}) ★"
            if row['Significativo'] else
            f"{var}\n(p = {row['p-valor']:.4f})"
        )

        axes[i].set_title(titulo, fontsize=10)
        axes[i].tick_params(axis='x', rotation=45)
        axes[i].grid(True, linestyle='--', alpha=0.5)

        diff = row['Diferença']
        axes[i].text(
            0.5, 0.95, f'Δ = {diff:.2f}',
            transform=axes[i].transAxes, ha='center',
            bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7),
            fontsize=8
        )

    except Exception as e:
        axes[i].set_title(f'Erro: {var}')
        print(f"Erro no plot de {var}: {e}")

# Remover eixos vazios
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.suptitle(
    'TODAS AS VARIÁVEIS — Significativas (★) primeiro, depois não significativas',
    fontsize=18, y=1.02
)

# Salvar esta figura no PDF
pdf.savefig(fig)
plt.close(fig)

# Fechar PDF
pdf.close()
print(f"\nPDF guardado como: {pdf_path}")



# 4. RESUMO FINAL
print("\n" + "="*80)
print("RESUMO FINAL DA ANÁLISE")
print("="*80)

print(f"\n1. AMOSTRAS:")
print(f"   • Grupo Alto (ros_p > 2000): {len(grupo_alto)} registros")
print(f"   • Grupo Baixo (ros_p ≤ 2000): {len(grupo_baixo)} registros")

print(f"\n2. SIGNIFICÂNCIA ESTATÍSTICA:")
print(f"   • Variáveis analisadas: {len(colunas_analise)}")
print(f"   • Variáveis significativas (p < 0.05): {len(df_significativas)}")
print(f"   • Taxa de significância: {len(df_significativas)/len(colunas_analise)*100:.1f}%")

# Top 10 variáveis por magnitude do efeito
print(f"\n3. TOP 10 VARIÁVEIS COM MAIOR IMPACTO:")
top_10_efeito = df_significativas.nlargest(10, 'abs_diferenca')
for i, (idx, row) in enumerate(top_10_efeito.iterrows(), 1):
    direcao = "MAIOR" if row['Diferença'] > 0 else "MENOR"
    print(f"   {i:2d}. {row['Variável']:20s} → {direcao} no grupo alto (Δ = {row['Diferença']:7.2f}, p = {row['p-valor']:.4f})")

# Padrões principais
print(f"\n4. PADRÕES PRINCIPAIS IDENTIFICADOS:")

padroes = {
    'VENTOS FORTES': ['wv100_k_av', 'wv10_kh_av', 'wv_950_av', 'wv_850_av', 'wv_700_av'],
    'CONDIÇÕES SECAS': ['sW_1m_av', 'sW_3m_av', 'sW_7_av', 'rh_2m_av', 'rh_700_av'],
    'ÍNDICES FOGO ALTOS': ['FWI_12h_av', 'DC_12h_av', 'VentIdx_av'],
    'ÉPOCA TARDIA': ['inidoy', 'enddoy'],
    'DURAÇÃO CURTA': ['duration_p'],
    'MENOR ELEVAÇÃO': ['elev_av']
}

for padrao, variaveis in padroes.items():
    vars_encontradas = [v for v in variaveis if v in df_significativas['Variável'].values]
    if vars_encontradas:
        print(f"   • {padrao}: {', '.join(vars_encontradas)}")

print("\n" + "="*80)
print("ANÁLISE CONCLUÍDA!")
print("="*80)

Preparando dados para análise...

Grupo com ros_p > 2000: 132 registros
Grupo com ros_p <= 2000: 1041 registros

Colunas para análise: 98

REALIZANDO TESTES ESTATÍSTICOS
Testes estatísticos concluídos!

CRIANDO RESUMO E VISUALIZAÇÕES
RESUMO DAS VARIÁVEIS MAIS IMPORTANTES (SIGNIFICATIVAS)

Variáveis com diferenças estatisticamente significativas (p < 0.05): 30

Top 20 variáveis mais importantes:
  Variável  Grupo Alto (média)  Grupo Baixo (média)  Diferença  p-valor
    HDW_av          39235.2030           25762.3231 13472.8799   0.0000
VentIdx_av          14851.7973           11410.7828  3441.0145   0.0000
 ros_p_lg1           2861.4344             755.5641  2105.8703   0.0000
gp_m2s2_av           3971.5785            4725.7786  -754.2001   0.0007
   f_start            962.1970            1670.5524  -708.3554   0.0000
  CBH_m_av           5136.8512            4532.2086   604.6426   0.0301
 VPD_Pa_av           2716.7144            2295.6867   421.0277   0.0007
 gp_300_av          93839.

In [4]:
# TABELA RESUMO COMPACTA
print("\n" + "="*100)
print("TABELA RESUMO - VARIÁVEIS SIGNIFICATIVAS (p < 0.05)")
print("="*100)

# Criar tabela compacta
tabela_compacta = df_significativas[['Variável', 'Grupo Alto (média)', 'Grupo Baixo (média)', 
                                     'Diferença', 'p-valor']].copy()

# Direção do efeito
tabela_compacta['Efeito'] = tabela_compacta['Diferença'].apply(
    lambda x: '↑ MAIOR' if x > 0 else '↓ MENOR'
)

# Diferença relativa (%)
tabela_compacta['Diferença Relativa (%)'] = (
    (tabela_compacta['Grupo Alto (média)'] - tabela_compacta['Grupo Baixo (média)']) /
     tabela_compacta['Grupo Baixo (média)']
) * 100

# Formatar números
for col in ['Grupo Alto (média)', 'Grupo Baixo (média)', 'Diferença']:
    tabela_compacta[col] = tabela_compacta[col].round(3)

tabela_compacta['Diferença Relativa (%)'] = tabela_compacta['Diferença Relativa (%)'].round(2)
tabela_compacta['p-valor'] = tabela_compacta['p-valor'].round(4)

# ✔ Reordenar colunas: Diferença e Diferença Relativa lado a lado
tabela_compacta = tabela_compacta[
    ['Variável',
     'Grupo Alto (média)', 
     'Grupo Baixo (média)',
     'Diferença',
     'Diferença Relativa (%)',
     'Efeito',
     'p-valor']
]

print(f"\nTotal de variáveis significativas: {len(tabela_compacta)}")

print("\nTabela ordenada por magnitude da diferença:")
print(
    tabela_compacta.to_string(index=False, max_colwidth=30)
)

# Salvar CSV
tabela_compacta.to_csv(r'..\..\Data\Data_Exploration\variaveis_significativas_ros_p_2000.csv', index=False, encoding='utf-8-sig')
print(f"\nTabela salva como 'variaveis_significativas_ros_p.csv'")



TABELA RESUMO - VARIÁVEIS SIGNIFICATIVAS (p < 0.05)

Total de variáveis significativas: 30

Tabela ordenada por magnitude da diferença:
  Variável  Grupo Alto (média)  Grupo Baixo (média)  Diferença  Diferença Relativa (%)  Efeito  p-valor
    HDW_av           39235.203            25762.323  13472.880                   52.30 ↑ MAIOR   0.0000
VentIdx_av           14851.797            11410.783   3441.014                   30.16 ↑ MAIOR   0.0000
 ros_p_lg1            2861.434              755.564   2105.870                  278.71 ↑ MAIOR   0.0000
gp_m2s2_av            3971.578             4725.779   -754.200                  -15.96 ↓ MENOR   0.0007
   f_start             962.197             1670.552   -708.355                  -42.40 ↓ MENOR   0.0000
  CBH_m_av            5136.851             4532.209    604.643                   13.34 ↑ MAIOR   0.0301
 VPD_Pa_av            2716.714             2295.687    421.028                   18.34 ↑ MAIOR   0.0007
 gp_300_av           93839.860 